<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/08_RAG.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 08. RAG

## Overview  
In this exercise, we will explore Retrieval-Augmented Generation (RAG) using the Solar framework. RAG combines retrieval-based techniques with generative models to improve the relevance and accuracy of generated responses by leveraging external knowledge sources. This notebook will guide you through implementing RAG and demonstrating its benefits in enhancing model outputs.
 
## Purpose of the Exercise
The purpose of this exercise is to integrate Retrieval-Augmented Generation into the Solar framework. By the end of this tutorial, users will understand how to use RAG to access external information and generate more informed and contextually accurate responses, thereby improving the performance and reliability of the language model.



## RAG: Retrieval Augmented Generation.
- Large language models (LLMs) have a limited context size.
- TLDR
- Not all context is relevant to a given question
- Query → Retrieve (Search) →  Results →  (LLM) →  Answer
- RAG is a method to combine LLM with Retrieval: Retrieval Augmented Generation



In [1]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25 python-dotenv tokenizers

In [2]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [3]:
from langchain_upstage import UpstageLayoutAnalysisLoader


layzer = UpstageLayoutAnalysisLoader(
    "pdfs/kim-tse-2008.pdf", use_ocr=False, output_type="html"
)
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [4]:
from IPython.display import display, HTML

display(HTML(docs[0].page_content[:1000]))

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [6]:
chain.invoke({"question": "What is bug classficiation?", "Context": docs})

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 32768 tokens. However, your messages resulted in 35940 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [7]:
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print(len(splits))



162


In [14]:
from transformers import AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

solar_tokenizer = AutoTokenizer.from_pretrained("upstage/solar-pro-preview-instruct")

token_splitter = TokenTextSplitter.from_huggingface_tokenizer(
    solar_tokenizer, chunk_size=250, chunk_overlap=100
)

splits = token_splitter.split_documents(docs)
print(len(splits))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


210


In [ ]:
retriever = BM25Retriever.from_documents(splits)
retriever.invoke("What is bug classficiation?")

[Document(page_content="<p id='111' style='font-size:18px'>One assumption of the presentation SO far is that a bug is<br>repaired in a single bug-fix change. What happens when a<br>bug is repaired across multiple commits? There are two<br>cases. In the first case, a bug repair is split across multiple<br>commits, with each commit modifying a separate section of<br>the code (code sections are disjoint). Each separate change is<br>tracked back to its initial bug-introducing change, which is<br>then used to train the SVM classifier. In the second case, a bug<br>fix occurs incrementally over multiple commits, with some<br>later fixes modifying earlier ones (the fix code partially<br>overlaps). The first patch in an overlapping code section<br>would be traced back to the original bug-introducing change.<br>Later modifications would not be traced back to the original<br>bug-introducing change. Instead, they would be traced back<br>to an intermediate modification, which is identified as bug",

In [ ]:
query = "What is bug classficiation?"
context_docs = retriever.invoke(query)
chain.invoke({"question": query, "Context": context_docs})

'The information is not present in the context.'

In [ ]:
query = "How do find the bug?"
context_docs = retriever.invoke(query)
chain.invoke({"question": query, "Context": context_docs})

'Bug classification is a process that involves classifying changes and predicting whether there is a bug in any of the lines that were changed in one file in one SCM commit transaction. It is different from bug prediction work that focuses on finding prediction or regression models to identify fault-prone or buggy modules, files, and functions. Bug classification allows for immediate bug predictions since it can predict buggy changes as soon as a change is made.'

# Excercise 
It seems keyword search is not the best for LLM queries. What are some alternatives?